# Define Event

Before we can analyze the event, we need to identify the boundaries of the event. We will accomplish this by

1. Defining the values to be classified as gains and losses
2. Creating a counts table defining the various events
3. Defining the porportion of patients with gain or loss to be considered significant
4. Identifying regions of gain and loss
5. Identifying regions where all cancer types meet criteria for gain or loss event.

## Setup

In [20]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import cptac
import numpy as np
import cnvutils

## Part 1: Define Parameters

These are the parameters that must be set for each analysis.

In [21]:
# # A list of cancer types to look at (should be all capital letter ex: BRCA)
# CANCER_TYPES = ['BRCA', 'COLON', 'HNSCC', 'LSCC', 'LUAD', 'OVARIAN']
# The porportion of patients with gain or loss of a given gene to be considered
PATIENT_CUTOFF = 0.15
# The chromosome to be analyzed (should be a string)
CHROMOSOME = '5'
# The arm to be analyzed. Options are: 'p', 'q' or 'both'.
ARM = 'both'

In [22]:
# Here you will need to load in all the cancer types to be considered. We have listed all for convenience, simply comment
# out the cancer types you do not want to consider.
cnv = {
    'BRCA': cptac.Brca().get_CNV(),
#     'CCRCC': cptac.Ccrcc().get_CNV(),
#     'COLON': cptac.Colon().get_CNV(),
#     'ENDO': cptac.Endometrial().get_CNV(),
#     'GBM': cptac.Gbm().get_CNV(),
    'HNSCC': cptac.Hnscc().get_CNV(),
    'LSCC': cptac.Lscc().get_CNV(),
    'LUAD': cptac.Luad().get_CNV(),
    'OVARIAN': cptac.Ovarian().get_CNV()
}

cptac warning: The HNSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (<ipython-input-22-c19e52eefc12>, line 9)


cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (<ipython-input-22-c19e52eefc12>, line 10)


## Part 2: Determine Cutoffs

In [23]:
counts = cnvutils.get_counts_table()

C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [24]:
cutoffs = dict()
for cancer_type in cnv.keys():
    df = cnv[cancer_type]
    cutoffs[cancer_type] = len(df) * PATIENT_CUTOFF

## Part 3: Subset Event

In [25]:
# counts.cancer.unique()

In [26]:
counts = counts[(counts.chromosome == CHROMOSOME)]
if ARM != 'both':
    counts = counts[counts.arm == ARM]

In [27]:
# counts

## Find Loss Regions

In [28]:
df = counts
loss_event_locations = dict()
for cancer in cnv.keys():
    df_loss = df[(df.variable == 'loss') & (df.cancer == cancer)].sort_values('start_bp')
    print(len(df_loss))
    values = list(df_loss.value)
    loss_events = list()
    start = None
    for i in range(0, len(values)):
        val = values[i]
        if val > cutoffs[cancer]:
            if start is None:
                start = i
        else:
            if start is not None:
                loss_events.append((start, i))
                start = None
    if start is not None:
        loss_events.append((start, len(values)-1))
    event_locations = list()
    for event in loss_events:
        start_bp = df_loss.iloc[event[0]].start_bp
        end_bp = df_loss.iloc[event[1]].start_bp
        event_locations.append((start_bp, end_bp-start_bp))
    loss_event_locations[cancer] = event_locations

976
2853
1003
818
759


In [29]:
loss_event_locations['LUAD']

[(86617928.0, 32218146.0), (180988846.0, 51379.0)]

In [30]:
loss_event_patients = list()
for cancer in loss_event_locations.keys():
    events = loss_event_locations[cancer]
    for event in events:
        start = event[0]
        end = event[0] + event[1]
        loss_event_patients.append((start, 1))
        loss_event_patients.append((end, 0))
#     patients += list(gain_event_locations[cancer])
loss_event_patients.sort()

In [31]:
counts

,Name,Database_ID,start_bp,end_bp,cancer,chromosome,arm,variable,value
17,AACSP1,ENSG00000250420.4,178764861.0,178818435.0,BRCA,5,q,gain,16
126,ABLIM3,ENSG00000173210.15,149141483.0,149260542.0,BRCA,5,q,gain,18
194,ACOT12,ENSG00000172497.4,81329996.0,81394179.0,BRCA,5,q,gain,10
224,ACSL6,ENSG00000164398.8,131949973.0,132012243.0,BRCA,5,q,gain,9
238,ACTBL2,ENSG00000169067.2,57480018.0,57482811.0,BRCA,5,q,gain,11
...,...,...,...,...,...,...,...,...,...
476203,YBX1P5,NaN,72417489.0,72418335.0,HNSCC,5,q,loss,31
476217,YTHDF1P1,NaN,97368776.0,97370421.0,HNSCC,5,q,loss,33
476517,ZNF346-IT1,NaN,177051714.0,177052963.0,HNSCC,5,q,loss,28
476579,ZRSR2P1,NaN,112891610.0,112893079.0,HNSCC,5,q,loss,33


In [32]:
count = 0
current_bp = 0
start = list()
end = list()
size = list()
total = list()
for patient in loss_event_patients:
    if patient[0] != current_bp:
        start.append(current_bp)
        end.append(patient[0])
        size.append(patient[0]-current_bp)
        total.append(count)
        current_bp = patient[0]
    if patient[1] == 1:
        count += 1
    else:
        count -= 1
loss_event_data = pd.DataFrame({'start': start, 'end': end, 'counts': total, 'length': size})

In [33]:
loss_event_data

,start,end,counts,length
0,0.0,38474668.0,0,38474668.0
1,38474668.0,38937920.0,1,463252.0
2,38937920.0,45254948.0,0,6317028.0
3,45254948.0,50396192.0,1,5141244.0
4,50396192.0,51383448.0,3,987256.0
5,51383448.0,86617928.0,4,35234480.0
6,86617928.0,118836074.0,5,32218146.0
7,118836074.0,148169733.0,4,29333659.0
8,148169733.0,149271859.0,3,1102126.0
9,149271859.0,150660874.0,4,1389015.0


## Find Gain Regions

In [34]:
df = counts
gain_event_locations = dict()
for cancer in cnv.keys():
    df_gain = df[(df.variable == 'gain') & (df.cancer == cancer)].sort_values('start_bp')
    values = list(df_gain.value)
    gain_events = list()
    start = None
    for i in range(0, len(values)):
        val = values[i]
        if val > cutoffs[cancer]:
            if start is None:
                start = i
        else:
            if start is not None:
                gain_events.append((start, i))
                start = None
    if start is not None:
        gain_events.append((start, len(values)-1))
    event_locations = list()
    for event in gain_events:
        start_bp = df_gain.iloc[event[0]].start_bp
        end_bp = df_gain.iloc[event[1]].start_bp
        
        event_locations.append((start_bp, end_bp-start_bp))
    gain_event_locations[cancer] = event_locations

In [35]:
gain_event_patients = list()
for cancer in gain_event_locations.keys():
    events = gain_event_locations[cancer]
    for event in events:
        start = event[0]
        end = event[0] + event[1]
        gain_event_patients.append((start, 1))
        gain_event_patients.append((end, 0))
#     patients += list(gain_event_locations[cancer])
gain_event_patients.sort()

In [36]:
count = 0
current_bp = 0
# results = list()
start = list()
end = list()
size = list()
total = list()
for patient in gain_event_patients:
    if patient[0] != current_bp:
        start.append(current_bp)
        end.append(patient[0])
        size.append(patient[0]-current_bp)
        total.append(count)
#         results.append((current_bp, patient[0], count))
        current_bp = patient[0]
    if patient[1] == 1:
        count += 1
    else:
        count -= 1
gain_event_data = pd.DataFrame({'start': start, 'end': end, 'counts': total, 'length': size})

In [37]:
gain_event_data

,start,end,counts,length
0,0.0,58198.0,0,58198.0
1,58198.0,92151.0,1,33953.0
2,92151.0,50396192.0,5,50304041.0
3,50396192.0,50665899.0,3,269707.0
4,50665899.0,51383448.0,2,717549.0
5,51383448.0,52787916.0,1,1404468.0
6,52787916.0,138561043.0,0,85773127.0
7,138561043.0,147585438.0,0,9024395.0
8,147585438.0,148025683.0,1,440245.0
9,148025683.0,148169733.0,2,144050.0
